In [10]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB,GaussianNB

In [13]:
df = pd.read_csv('train.csv',index_col=0)

In [14]:
df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
id,,,,,,,,,,,,,,,,,,,,,
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,44,93,15.4,1,0.8,19,13,17,0,1


In [23]:
X = df.drop('smoking',axis=1)
y = df['smoking']

In [24]:
X_train , X_test , y_train ,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [29]:
ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore').set_output(transform='pandas')
gb = GaussianNB()
pipe = Pipeline([('OHE',ohe),('GB',gb)])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.90      0.74     26881
           1       0.72      0.34      0.46     20896

    accuracy                           0.65     47777
   macro avg       0.68      0.62      0.60     47777
weighted avg       0.67      0.65      0.62     47777



In [31]:
y_pred_proba = pipe.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test,y_pred_proba))

0.7751337873967581


In [32]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [33]:
params = {'GB__var_smoothing':np.linspace(1e-15,1,10)}

In [34]:
gcv = GridSearchCV(pipe,param_grid=params,scoring='roc_auc',cv=kfold)

In [35]:
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('GB', GaussianNB())]),
             param_grid={'GB__var_smoothing': array([1.00000000e-15, 1.11111111e-01, 2.22222222e-01, 3.33333333e-01,
       4.44444444e-01, 5.55555556e-01, 6.66666667e-01, 7.77777778e-01,
       8.88888889e-01, 1.00000000e+00])},
             scoring='roc_auc')

In [36]:
gcv.best_params_

{'GB__var_smoothing': 0.111111111111112}

In [37]:
gcv.best_score_

0.8030531402229281